# NYC Rent

**GOAL:** Improve our model for predicting NYC rent prices.

**Objectives**

- Do one-hot encoding of categorical features
- Do univariate feature selection
- Use scikit-learn to fit Ridge Regression models

In [104]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# I. Wrangle Data

In [105]:
import numpy as np
import pandas as pd

def wrangle(filepath):
  # import the CSV file
  df = pd.read_csv(filepath, 
                   parse_dates=['created'],
                   index_col='created')
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
          (df['price'] <= np.percentile(df['price'], 99.5)) & 
          (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
          (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
          (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
          (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
  
  # Drop high-cardinality categorical variables
  # df.drop(columns= ['description', 'display_address', 'street_address'], inplace=True)

# function to drop the columns practice PS
  drop_cols = [col for col in df.select_dtypes('object') if df[col].nunique() > 10]
  df.drop(columns = drop_cols, inplace=True)

#   # Tori's proposal
#   drop_cols = [col for col in df.select_dtypes('object')
#               if df[col].nunique() > 10]
#   df.drop(columns=drop_cols, inplace=True)

  return df

df = wrangle(DATA_PATH + 'apartments/renthop-nyc.csv')

In [106]:
df.shape

(48817, 30)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 48817 entries, 2016-06-24 07:54:24 to 2016-04-12 02:48:07
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48817 non-null  float64
 1   bedrooms              48817 non-null  int64  
 2   latitude              48817 non-null  float64
 3   longitude             48817 non-null  float64
 4   price                 48817 non-null  int64  
 5   interest_level        48817 non-null  object 
 6   elevator              48817 non-null  int64  
 7   cats_allowed          48817 non-null  int64  
 8   hardwood_floors       48817 non-null  int64  
 9   dogs_allowed          48817 non-null  int64  
 10  doorman               48817 non-null  int64  
 11  dishwasher            48817 non-null  int64  
 12  no_fee                48817 non-null  int64  
 13  laundry_in_building   48817 non-null  int64  
 14  fitness_center        48817 non-nul

In [108]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [109]:
# find out the unique values for any column
df.elevator.unique()
# 0 and 1 denote are a sample of One Hot Coding
df.interest_level.value_counts() 
# interest level is categorical
# df.display_address.nunique() # high cardinality category variables # column dropped now

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [110]:
df.select_dtypes('object').head()

,interest_level
created,
2016-06-24 07:54:24,medium
2016-06-12 12:19:27,low
2016-04-17 03:26:41,high
2016-04-18 02:22:02,low
2016-04-28 01:32:41,low


In [111]:
df.select_dtypes('object').nunique()
# description, street_address, street_address all these are example of high cardinality categorical variables

interest_level    3
dtype: int64

Categorical variables from above are elevator, cats_allowed, hardwood_floors..... etc. (all perks)

# II. Split Data

Split **target vector** from **feature matrix**.

In [112]:
target = 'price'
y = df[target]
X = df.drop(columns=target)

Split data into **training** and **test** sets.

In [113]:
cutoff = '2016-06-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

# III. Establish Baseline

**Note:** This is a **regression** problem because we're predictiong the continuous value `'price'`.

In [114]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 1201.8811133682555


# IV. Build `LinearRegression` Model

**NOTES**
  
  - Before we train our model, we need to **encode** our categorical features.
  - For high-dimentional data, your have a few options. First is **dimentionality reduction** using `SelectKBest`.

In [137]:
# Step 1: import classes
from sklearn.linear_model import LinearRegression, Ridge
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest


In [134]:
# Step 2: Instantiate our predictor
transformer_1 = OneHotEncoder(use_cat_names=True)
transformer_2 = SelectKBest(k=25) # k is a HYPERPARAMETER

# Step 3: fit the transformer to the model data
# NEVER refit your transformer when transforming your TEST data
transformer_1.fit(X_train)

# Step4: Transform the data
XT_train = transformer_1.transform(X_train)
XT_test = transformer_1.transform(X_test)

# Step:
transformer_2.fit(XT_train, y_train)

# Step:
XTT_train = transformer_2.transform(XT_train)
XTT_test = transformer_2.transform(XT_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [151]:
# Step 2: Instantiate the PREDICTOR
model = LinearRegression()

# Step 3: Fit the model on training data
model.fit(XTT_train, y_train) # make sure you have encoded with the categorical variable

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Another option is to use a predictor that does some sort of **regularization**. 

In [152]:
model_r = Ridge(alpha= 1.0) # alpha is also a HYPERPARAMETER --> HYPERPARAMETER is what we set before fit

model_r.fit(XT_train, y_train) # this is the OHE data with no selectKBest


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# V. Check Metrics

In [153]:
print('LR Training MAE: ', mean_absolute_error(y_train, model.predict(XTT_train)))
print('LR Test MAE: ', mean_absolute_error(y_test, model.predict(XTT_test)))

LR Training MAE:  673.3136812759128
LR Test MAE:  676.4400492000973


In [154]:
print('Ridge Training MAE: ', mean_absolute_error(y_train, model_r.predict(XT_train)))
print('Ridge Test MAE: ', mean_absolute_error(y_test, model_r.predict(XT_test)))

Ridge Training MAE:  672.6640726589972
Ridge Test MAE:  675.514908956413


# Communicate results

More on this tomorrow...